In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


In [2]:
#Read our Dataset
df = pd.read_csv("C:\\Users\\Uchenna\\Desktop\\my work\\world_cup_results.csv")

In [3]:
#Initial View of our dataframe shape
df.shape

(852, 11)

In [4]:
#Some information about our dataframe shows we have no missing values
#However our date field is nit of the right type. we'll dealwith it as we go
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         852 non-null    int64 
 1   Date         852 non-null    object
 2   Time         852 non-null    object
 3   Round        852 non-null    object
 4   Stadium      852 non-null    object
 5   City         852 non-null    object
 6   HomeTeam     852 non-null    object
 7   HomeGoals    852 non-null    int64 
 8   AwayGoals    852 non-null    int64 
 9   AwayTeam     852 non-null    object
 10  Observation  852 non-null    object
dtypes: int64(3), object(8)
memory usage: 73.3+ KB


In [5]:
#Depending on your data source, your might have column n=name with spaces
#One quick thing I do most times is to convert everything to lowercase and ass am underscore in place of space
#It's not the case here but I will drop in the flow all the same
#headers = [line.lower().replace('','_') for line in df.columns]
#df.columns = headers
#df.head()

In [6]:
#Let's deal with the duplicates
#Notice that from a shape of (852cols, 11rows) we now arrive at (836, 11) there was some duplicates
df = df.drop_duplicates()
df.shape

(836, 11)

In [66]:
#Total goals column
df['TotalGoals'] = df['AwayGoals'] + df['HomeGoals']

In [87]:
#Get a day of the week and month column
#split the literal string, pick the 2nd item
df['day'] = pd.to_datetime(df['Date']).dt.day_name()#convert to datetime and get day of the week
 

KeyError: 'Date'

In [54]:
df = df[['Year', 'day', 'Time','Round','HomeTeam', 'AwayTeam', 'HomeGoals', 'AwayGoals','TotalGoals']]
df.head(2)

,Year,day,Time,Round,HomeTeam,AwayTeam,HomeGoals,AwayGoals,TotalGoals
0,1930,Saturday,15:00,Group 1,France,Mexico,4,1,5
1,1930,Saturday,15:00,Group 4,USA,Belgium,3,0,3


In [55]:
df.head(2)

,Year,day,Time,Round,HomeTeam,AwayTeam,HomeGoals,AwayGoals,TotalGoals
0,1930,Saturday,15:00,Group 1,France,Mexico,4,1,5
1,1930,Saturday,15:00,Group 4,USA,Belgium,3,0,3


In [56]:
#Question1 How many matches wereplayed each World cup from 1930
#A value_count on the Year column nicely delivers thos
#To plot this effortlessly with plotly, we will convert the results to a fresh dataframe
#Notice how nicely plotly highlights the expected years world cups were mot played
matches_per_year = df.Year.value_counts() #a series into a dataframe
all_games = pd.DataFrame(matches_per_year) #mske series into dataframe
all_games.reset_index (inplace=True) #reset index inplace
all_games.columns = ['Year', 'Matches'] #rename the columns as needed
fig = px.bar(all_games, x='Year', y='Matches', text ='Matches', color = 'Matches', height = 500,
            labels={'Matches': 'Matches Played', 'Year':'World Cup Year'},
            title = "Total Matches Played in Each World Cup Year")
fig.update_traces(texttemplate = '%{text}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_xaxes(
        tickangle=45, tickfont=dict(family='Arial', color='blue', size=14), 
        tickvals=[line for line in all_games.Year])
fig.show()

In [57]:
#Question2: Total goals scored for each Tournament Year
all_goals = df.groupby(['Year']).TotalGoals.sum()
all_goals_df = pd.DataFrame(all_goals)
all_goals_df.reset_index(inplace=True) #reset it's index inplace
all_goals_df.columns = ['Year', 'Goals'] #rename the columns as needed

fig = px.bar(all_games, x='Year', y='Goals', text ='Goals', color = 'Goals', height = 480,
         labels={'Goals': 'Goals Scored', 'Year':'World Cup Year'},
            title = "Total Goals Scored in Each World Cup Year")
fig.update_traces(texttemplate = '%{text}', textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_xaxes(
        tickangle=45, tickfont=dict(family='Arial', color='blue', size=14), 
        tickvals=[line for line in all_goals.Year])
fig.show()


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['Year', 'Matches'] but received: Goals

In [58]:
#Questions3: All teams who have reached finals and how mant times
#Take a piece of the df corresponding to all "Final" in the 'Round' Column
all_finals = df[df['Round'] == 'Final']
all_finals.head()

,Year,day,Time,Round,HomeTeam,AwayTeam,HomeGoals,AwayGoals,TotalGoals
17,1930,Tuesday,14:15,Final,Uruguay,Argentina,4,2,6
34,1934,Saturday,17:30,Final,Italy,Czechoslovakia,2,1,3
52,1938,Saturday,17:00,Final,Italy,Hungary,4,2,6
100,1954,Saturday,17:00,Final,Germany FR,Hungary,3,2,5
135,1958,Saturday,15:00,Final,Brazil,Sweden,5,2,7


In [59]:
#Let's make a list of all teams who reached this stage
#This will be a list of all featuring HomeTeams and AwayTeams
#A simple concatenation of a python list of both will do
teams = [line for line in all_finals.HomeTeam] + [line for line in all_finals.AwayTeam]
teams[:5]

['Uruguay', 'Italy', 'Italy', 'Germany FR', 'Brazil']

In [60]:
#To Count the frequency that I am going to eventually plot I prefer to use a dataframe. It's seamless
#So I will make a dataframe from the list 'teams' and take a drop a value_counts(). Neat, yea?
all_finals_df = pd.DataFrame(columns=['Teams'], data = teams)
all_finals_df.head(3)

,Teams
0,Uruguay
1,Italy
2,Italy


In [61]:
#To demonstrate this value_count() counting, see the results before we plot
#PS: I am choosing to leave Germany and Germany FR as different teams 
all_finals_df.Teams.value_counts()

Brazil            6
Italy             6
Germany FR        6
Argentina         5
Netherlands       3
Hungary           2
Germany           2
Czechoslovakia    2
France            2
England           1
Spain             1
Uruguay           1
Sweden            1
Name: Teams, dtype: int64

In [23]:
#Let me get this same result outside pandas
finals_teams = {} #Dict to team as key and apperance number as value
for team in teams:
    if team in finals_teams.keys(): #if this is already in the dict
        finals_teams[team] += 1 #increment it by one
    else: #else
            finals_teams[team] = 1 #it's first instance in the loop, assign it a value of 1
            
finals_teams

{'Uruguay': 1,
 'Italy': 6,
 'Germany FR': 6,
 'Brazil': 6,
 'England': 1,
 'Netherlands': 3,
 'Argentina': 5,
 'Germany': 2,
 'Czechoslovakia': 2,
 'Hungary': 2,
 'Sweden': 1,
 'France': 2,
 'Spain': 1}

In [24]:
#We know that python dictionary as a property is unordered so we can't successfully sort a regular dict
#The above is not sorted so let's sort and reverse to get it in descending order.
#Notice that the result is a list of tuples. To plot take not of this.
#Compare the result below with what value_counts() gave us above
#I will prefer to plot with a dataframe made from value_counts()
sorted(finals_teams.items(),key=lambda x:x[1], reverse=True)

[('Italy', 6),
 ('Germany FR', 6),
 ('Brazil', 6),
 ('Argentina', 5),
 ('Netherlands', 3),
 ('Germany', 2),
 ('Czechoslovakia', 2),
 ('Hungary', 2),
 ('France', 2),
 ('Uruguay', 1),
 ('England', 1),
 ('Sweden', 1),
 ('Spain', 1)]

In [26]:
#Make a dataframe from counting values in all_finals_df
finals_teams_ranked = all_finals_df.Teams.value_counts()
finals_teams_ranked_df = pd.DataFrame(finals_teams_ranked)
finals_teams_ranked_df.reset_index(inplace=True)
finals_teams_ranked_df.columns = ['Teams', 'Frequency'] #rename the coloumn as needed

fig = px.bar(finals_teams_ranked_df, x = 'Teams', y = 'Frequency', color='Frequency', height=450,
             labels={'Teams': 'Teams in the Finals'},
            title = "All Temas Who Have Reached Finals and Frequency")
fig.update_layout(uniformtext_minsize = 8)
fig.show()

In [68]:
#Question 4: All teams who have reached Semi Final and how many times
#Take a piece of the df corresponding to all "semi-finals" in the round column
all_semi_finals = df[df['Round'] == 'Semi-finals']
all_semi_finals.head()

,Year,day,Time,Round,HomeTeam,AwayTeam,HomeGoals,AwayGoals,TotalGoals
15,1930,Friday,14:45,Semi-finals,Argentina,USA,6,1,7
16,1930,Saturday,14:45,Semi-finals,Uruguay,Yugoslavia,6,1,7
31,1934,Saturday,16:30,Semi-finals,Italy,Austria,1,0,1
32,1934,Saturday,16:30,Semi-finals,Czechoslovakia,Germany,3,1,4
49,1938,Wednesday,18:00,Semi-finals,Hungary,Sweden,5,1,6


In [69]:
#Make a list of all teams invovled Home and Away
teams = [line for line in all_semi_finals.HomeTeam] + [line for line in all_semi_finals.AwayTeam]

In [70]:
#Makre a df of teams
all_semi_finals_df = pd.DataFrame(columns=['Teams'], data = teams)
#peep the head()
all_semi_finals_df.head(2)


,Teams
0,Argentina
1,Uruguay


In [71]:
#Make a dataframe from counting values in all semi_finals
#See 'Finals' cell above for explanation as the steps are identical. We are avoidin functions for practice
semifinals_teams_ranked = all_semi_finals_df.Teams.value_counts()
semifinals_teams_ranked_df = pd.DataFrame(semifinals_teams_ranked)
semifinals_teams_ranked_df.reset_index(inplace=True)
semifinals_teams_ranked_df.columns = ['Teams' , 'Frequency']
fig = px.bar(finals_teams_ranked_df, x = 'Teams', y = 'Frequency', color='Frequency', height=450,
             labels={'Teams': 'Teams in the Finals'},
            title = "All Temas Who Have Reached Semi-Finals and Frequency")
fig.update_layout(uniformtext_minsize = 8)
fig.show()

In [83]:
#Question 5: How many goals and average goals scored in all Semi-Finals
#Let's take a sum and mean of "TotalGoals"
semi_goals_sum = all_semi_finals.TotalGoals.sum()
semi_goals_ave = all_semi_finals.TotalGoals.mean()
print(f" {semi_goals_sum} goals were scored in all Semi-Finals\nAn average of {semi_goals_ave:.2f} in every match.")

 123 goals were scored in all Semi-Finals
An average of 3.62 in every match.


In [73]:
#Question6 : How Many Goals and Average goals scored in all Quarter-finals
all_qrts = df[df['Round'] == 'Quarter-finals']
all_qrts.head(3)

,Year,day,Time,Round,HomeTeam,AwayTeam,HomeGoals,AwayGoals,TotalGoals
26,1934,Wednesday,16:30,Quarter-finals,Czechoslovakia,Switzerland,3,2,5
27,1934,Wednesday,16:30,Quarter-finals,Germany,Sweden,2,1,3
28,1934,Wednesday,16:30,Quarter-finals,Italy,Spain,1,1,2


In [82]:
qrts_goals_sum = all_qrts.TotalGoals.sum()
qrts_goals_ave = all_qrts.TotalGoals.mean()
print(f" {qrts_goals_sum} goals were scored in all Semi-Finals\nAn average of {qrts_goals_ave:.2f} in every match.")

 175 goals were scored in all Semi-Finals
An average of 2.82 in every match.


In [79]:
#All goals in finals
finals = df[df['Round'] == 'Final']['HomeGoals'].sum() + df[df['Round'] == 'Final']['AwayGoals'].sum()
print(f"{finals} goals in finals")

68 goals in finals


In [81]:
#How many goals and Average number scored in all finals
#Working with the piece of the original df holding finals

finals_goals_sum = all_finals.TotalGoals.sum()
finals_goals_ave = all_finals.TotalGoals.mean()
print(f"{finals_goals_sum} goals were scored in all Finals\nAn average of {finals_goals_ave:.2f} in every match.")

68 goals were scored in all Finals
An average of 3.58 in every match.


In [84]:
#How many matches were played outside quarter-finals and above
#Let's be creative here
#First get a slice with no finals
df_less_finals = df[df['Round'] != 'Finals']

#From there get a slices with no semi-finals and we are left with all matches neither finals nor semi-finals 
df_less_finals_semi = df_less_finals[df_less_finals['Round'] != 'semi-Finals']

#One more dropping qrts. 
df_less_finals_semi_qrts = df_less_finals_semi[df_less_finals_semi['Round'] != 'Quarter-Finals']

In [86]:
#Did it work? Let's find out

'Final' in df_less_finals_semi_qrts.Round.tolist() or 'Semi-Finals' in df_less_finals_semi_qrts.Round.tolist()

True

In [88]:
#Just in Case that slice was lay-luck, Let's make sure other Roundare there
'Round of 16' in df_less_finals_semi_qrts.Round.tolist()

True

In [93]:
#Total Matches in the slice of the dataframe is same number of rows. A number of move will show the number 
d_rest0 = df_less_finals_semi_qrts.shape[0]
d_rest1 = len(df_less_finals_semi_qrts)

d_rest0 == d_rest1

True

In [94]:
print(f"There are {d_rest0} matches played outside Quarter-finals and above")

There are 836 matches played outside Quarter-finals and above


In [ ]:
#Kickers 